# 地理情報分析

In [ ]:
# このセルはそのまま実行して、tdquiz マジックコマンドをロードしてください。
# tdquiz コマンドはクエリを実行してその結果を返します。
%load_ext tdquiz

# tdquiz コマンドをテスト
%tdquiz SELECT USER, SESSION, CURRENT_TIMESTAMP

<h3>２つのオフィス間の距離</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">distance</span>
<br>
<em>facilities.teradata_offices</em> テーブルには、Teradata社のオフィスの所在地情報が格納されています。そのうち２つは日本のオフィスです。
これら２つのオフィス間の距離を km 単位で計算してください。
ただし、距離計算には<a href="https://ja.wikipedia.org/wiki/%E5%A4%A7%E5%86%86%E8%B7%9D%E9%9B%A2" rel="noopener noreferrer" target="_blank">大円距離公式</a>を用いること。
<details><summary>Hint</summary><ul>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types-17.20/ST_Point-Methods/ST_SphericalDistance" rel="noopener noreferrer" target="_blank">Geospatial Data Types (マニュアル)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  (NEW ST_Geometry('ST_Point', a.longitude, a.latitude)).ST_SphericalDistance(
    NEW ST_Geometry('ST_Point', b.longitude, b.latitude)) / 1000 AS distance_km
FROM
  facilities.teradata_offices AS a
  INNER JOIN facilities.teradata_offices AS b
    ON a.office_name < b.office_name
       AND a.country = 'jpn'
       AND b.country = 'jpn'</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>村にある新幹線駅</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">join</span>
<br>
新幹線の駅のうちで村にあるものをすべて抽出し、その路線名、都道府県名、村名、駅名を表示してください。
<details><summary>Hint</summary><ul>
<li><em>japan.cities</em>, <em>facilities.stations</em>, <em>facilities.shinkansen</em>
<li>図形が「含まれている」に該当する関数は？
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods" rel="noopener noreferrer" target="_blank">Geometry methods (マニュアル)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  s.line_name,
  c.pref_name,
  c.city_name,
  s.station_name
FROM
  facilities.stations AS s
  INNER JOIN japan.cities AS c
    ON c.geometry.ST_INTERSECTS(s.geometry) = 1
WHERE 
  s.line_name IN ( SELECT DISTINCT line_name FROM facilities.shinkansen )
  AND RIGHT(c.city_name, 1) = '村'</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>最果ての駅</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span>
<br>
日本最北端、最南端、最西端、最東端の駅はどこですか？
<details><summary>Hint</summary><ul>
<li><em>ST_X, ST_Y</em> を調べてみてください。
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods" rel="noopener noreferrer" target="_blank">Geometry methods (マニュアル)</a>
<li><a href="https://tabi-mag.jp/station-news/" rel="noopener noreferrer" target="_blank">よければ解いた後に</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">with ends AS (
  SELECT
    MIN(geometry.ST_X()) AS east_end,
    MAX(geometry.ST_X()) AS west_end,
    MIN(geometry.ST_Y()) AS south_end,
    MAX(geometry.ST_Y()) AS north_end
  FROM
    facilities.stations
)
SELECT
  north.station_name AS northmost,
  south.station_name AS southmost,
  west.station_name AS westmost,
  east.station_name AS eastmost
FROM
  ends
  INNER JOIN facilities.stations AS east ON east.geometry.ST_X() = ends.east_end
  INNER JOIN facilities.stations AS west ON west.geometry.ST_X() = ends.west_end
  INNER JOIN facilities.stations AS north ON north.geometry.ST_Y() = ends.north_end
  INNER JOIN facilities.stations AS south ON south.geometry.ST_Y() = ends.south_end</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>新幹線が止まらない都道府県</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">join</span>
<br>
都道府県のうち新幹線が通るものを抽出し、その停車駅の数を計算してください。通過するだけで停車はしない都道府県ははありますか？
結果は 都道府県名と新幹線駅の数として、駅の数と都道府県名について降順に並べてください。
<details><summary>Hint</summary><ul>
<li><em>japan.cities</em>, <em>facilities.stations</em>, <em>facilities.shinkansen</em>
<li>図形が「含まれている」に該当する関数は？
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods" rel="noopener noreferrer" target="_blank">Geometry methods (マニュアル)</a>
<li><a href="https://news.mynavi.jp/article/trivia-435/" rel="noopener noreferrer" target="_blank">よければ解いた後に</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH tmp AS (
  SELECT DISTINCT
    p.pref_name
  FROM
    japan.prefectures AS p
    INNER JOIN facilities.shinkansen AS s
      ON p.geometry.ST_INTERSECTS(s.geometry) = 1
),
tmp2 AS (
  SELECT
    p.pref_name, count(*) AS num_stations
  FROM
    facilities.stations AS s
    INNER JOIN japan.prefectures AS p
      ON p.geometry.ST_INTERSECTS(s.geometry) = 1
  WHERE
    s.line_name IN ( SELECT DISTINCT line_name FROM facilities.shinkansen )
  GROUP BY 1
)
SELECT
  a.pref_name,
  COALESCE(b.num_stations, 0) AS num_stations
FROM
  tmp AS a LEFT JOIN tmp2 AS b ON a.pref_name = b.pref_name
ORDER BY 2,1</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>飛び地の市区町村</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">geospatial</span>
<br>
地方自治体の中には、何らかの事情で飛び地が存在する（直接つながらない複数の部分に分かれている）ものがあります。
群馬県に存在する飛び地の市区町村の名前をすべて表示してください。
<details><summary>Hint</summary><ul>
<li><em>japan.cities</em> を使います
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods/ST_GeometryType-Method" rel="noopener noreferrer" target="_blank">ST_GeometryType (マニュアル)</a>
<li>複数に分かれた幾何情報タイプとは？
<li><a href="https://ja.wikipedia.org/wiki/%E9%A3%9B%E5%9C%B0" rel="noopener noreferrer" target="_blank">飛び地 (Wikipedia)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  pref_name, city_name
FROM
  japan.cities
WHERE
  pref_name = '群馬県'
  AND geometry.ST_GeometryType() = 'ST_MultiPolygon'</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */






<h3>地理情報列の一覧
</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">teradata-specific</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">ddl</span>
<br>
<em>japan</em>, <em>facilities</em>, <em>TeraShirt</em> データベースに含まれるテーブルについて、地理情報型 (ST_Geometry) の列を全て抽出してください。
結果には、データベース名、テーブル名、地理情報列名を表示してください。
<details><summary>Hint</summary><em>dbc.ColumnsV</em> を調べてみてください。</details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  databaseName, tableName, columnName
FROM
  dbc.ColumnsV
WHERE
  columnUDTName = 'ST_GEOMETRY'
  AND databaseName IN ('japan', 'facilities', 'TeraShirt')</code></pre></details>

In [ ]:
%%tdquiz
/* クエリを下記に記載 */




